# **Preliminaries**

Before diving into _this_ tutorial, I would like to suggest having a look at this [link](https://github.com/gtolomei/python-for-datascience#Class-Schedule) to those who are not too familiar with **Python** and **Jupyter Notebook**. In particular, I would recommend them going through _Lecture 1_ to _5_ (included), as those cover most of the material needed to understand the basics of the Python programming language and the Jupyter Notebook environment (which is very similar to Google Colab).

# **PySpark + Colab Setup**

In [0]:
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64" # Set path to JAVA_HOME

## **1.** Install PySpark and related dependencies

In [0]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = JAVA_HOME

## **2.** Import useful PySpark packages

In [0]:
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

## **3.** Create Spark context

In [0]:
# Create the session
conf = SparkConf().set("spark.ui.port", "4050").set('spark.executor.memory', '4G').set('spark.driver.memory', '45G').set('spark.driver.maxResultSize', '10G')

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

## **4.** Check everything is ok

In [0]:
spark

In [0]:
sc._conf.getAll()

# **Create PySpark's <code>DataFrame</code> "manually"**

Let's first create <code>Employee</code> and <code>Department</code> <code>Row</code> instances.

In [0]:
# Row is a generic "record" object with an ordered collection of fields that can be accessed by index or name
# In this case, we use Row to create the schema of other Row objects
Employee = Row("firstName", "lastName", "email", "salary")

# Create some Row instances following the above schema
employee_1 = Employee('Basher', 'armbrust', 'bash@edureka.co', 100000)
employee_2 = Employee('Daniel', 'meng', 'daniel@stanford.edu', 120000 )
employee_3 = Employee('Muriel', None, 'muriel@waterloo.edu', 140000 )
employee_4 = Employee('Rachel', 'wendell', 'rach_3@edureka.co', 160000 )
employee_5 = Employee('Zach', 'galifianakis', 'zach_g@edureka.co', 160000 )

# Print out a whole Row instance
print(employee_3)

In [0]:
# Access and print out the first Row field of `employee_1` by index
print("Employee 1's first name is: {:s}".format(employee_1[0]))

# Access and print out the `salary` Row field of `employee_4` by name
print("Employee 4's salary is: {:d}".format(employee_4.salary))

In [0]:
# Create another set of Row objects. 
# This time, schema is defined by means of 2 named attributes, i.e., `id` and `name`.
# Each Row object specifies those attribute at creation time 
# (rather than having a `Department` Row object containing only the schema, as we did before with `Employee`)
department_1 = Row(id='123456', name='HR')
department_2 = Row(id='789012', name='OPS')
department_3 = Row(id='345678', name='FN')
department_4 = Row(id='901234', name='DEV')

Now, we'll create a <code>DepartmentWithEmployees</code> <code>Row</code> instance from the Employee and Departments.

In [0]:
departmentWithEmployees_1 = Row(department=department_1, employees=[employee_1, employee_2, employee_5])
departmentWithEmployees_2 = Row(department=department_2, employees=[employee_3, employee_4])
departmentWithEmployees_3 = Row(department=department_3, employees=[employee_1, employee_4, employee_3])
departmentWithEmployees_4 = Row(department=department_4, employees=[employee_2, employee_3])

In [0]:
print(departmentWithEmployees_2)

Let's create our <code>DataFrame</code> from the list of rows above

In [0]:
departmentsWithEmployees = [departmentWithEmployees_1, 
                            departmentWithEmployees_2, 
                            departmentWithEmployees_3,                           
                            departmentWithEmployees_4]

# Create the Spark's DataFrame object
df = spark.createDataFrame(departmentsWithEmployees)
# Print out the DataFrame
df.show(n=2, truncate=False) # try, playing with the arguments (e.g., truncate=True/False)

# **Create PySpark's <code>DataFrame</code> from an input source**

Most of the times, data we will be working with are not manually created as above, yet they are stored in various formats like <code>csv</code>, <code>json</code>, <code>xml</code>, or a [Parquet](https://spark.apache.org/docs/latest/sql-data-sources-parquet.html) file. Moreover, data can be loaded either from _local_ or _distributed_ file systems (e.g., HDFS or Amazon S3).

Remember, though, that our development environment is Google Colab, which - differently from Jupyter Notebook that usually runs on our local machine - itself runs on the (Google's) cloud infrastructure. As such, "_local_" file system **in this case** means local to Google's cloud! In other words, any references to data made within Google Colab is relative to the Google's end side. That is why we need Google's cloud to know how to get our data before we can make use of it.

Roughly, there are **2 possible ways** of doing this:
- "pushing" data to Google Drive;
- using an external data stores like Amazon S3.

In this class, we will be using the first option, although in very large, real world production environments the second method will be the preferred way to go.

**Additional References:** 
- More information on how to let Google Colab know how to get data can be found [here](https://towardsdatascience.com/importing-data-to-google-colab-the-clean-way-5ceef9e9e3c8)
- An interesting discussion on the performance of various input formats (mostly CSV vs. Parquet) can be found [here](https://towardsdatascience.com/a-brief-introduction-to-pyspark-ff4284701873)

## **1. Link Google Colab to our Google Drive**

In [0]:
GDRIVE_DIR = "/content/gdrive" # Your own mount point on Google Drive
GDRIVE_HOME_DIR = GDRIVE_DIR + "/My Drive" # Your own home directory
GDRIVE_DATA_DIR = GDRIVE_HOME_DIR + "/Teaching/2019-20-BDC/datasets" # Your own data directory

In [0]:
# Point Colaboratory to our Google Drive
from google.colab import drive

drive.mount(GDRIVE_DIR, force_remount=True)

## **2. Data Acquisition**

Let's see how to download/store a dataset file located at a remote source directly to Google Drive.

In [0]:
DATASET_URL = "https://github.com/gtolomei/big-data-computing/raw/master/datasets/fifa-players-2020.csv.bz2"
GDRIVE_DATASET_FILE = GDRIVE_DATA_DIR + "/" + DATASET_URL.split("/")[-1]

### **Download dataset file from URL directly to our Google Drive**

In [0]:
import requests

"""
This function downloads a file from a specific URL directly to Google Drive.
"""
def get_data(dataset_url, dest, chunk_size=1024):
  response = requests.get(dataset_url, stream=True)
  if response.status_code == 200: # Test if everything went ok
    with open(dest, "wb") as file:
      for block in response.iter_content(chunk_size=chunk_size): 
        if block: 
          file.write(block)

In [0]:
print("Retrieving dataset from URL: {} ...".format(DATASET_URL))
get_data(DATASET_URL, GDRIVE_DATASET_FILE)
print("Dataset successfully retrieved and stored at: {}".format(GDRIVE_DATASET_FILE))

### **Read dataset file into a Spark Dataframe**


In [0]:
fifa_df = spark.read.load(GDRIVE_DATASET_FILE, 
                           format="csv", 
                           sep=",", 
                           inferSchema="true", 
                           header="true")

### **Display the first <code>n=5</code> rows of the loaded dataset**

In [0]:
fifa_df.show(n=5, truncate=False)

### **Print out the schema of the DataFrame**

In [0]:
fifa_df.printSchema()

### **Check the shape of the loaded dataset, i.e., number of rows and columns**

In [0]:
print("The shape of the dataset is {:d} rows by {:d} columns".format(fifa_df.count(), len(fifa_df.columns)))

### **Check the data types of the loaded dataset**

In [0]:
print(fifa_df.dtypes)

### **Describing a particular column**

If we want to have a look at the summary of any particular column of a DataFrame, we use the <code>describe</code> method. This method gives us the statistical summary of the given column if not specified, it provides the statistical summary of the DataFrame.

In [0]:
# Statistical summary of the `age` column
fifa_df.describe("age").show()

In [0]:
# Statistical summary of the `height_cm` column
fifa_df.describe("height_cm").show()

## **3. Data Manipulation**

Once data has been successfully loaded into a Spark DataFrame, we can start working with it. Most of the operations involve: _selecting_, _filtering_, _sorting_, _grouping_, and compute _aggregate functions_ (e.g., _count_).

### **Selecting Multiple Columns**

If we want to select particular columns from the DataFrame, we use the <code>select</code> method.

In [0]:
fifa_df.select(["long_name", "club", "nationality"]).show(n=10, truncate=False)
# Alternatively (no list notation):
# fifa_df.select("long_name", "club", "nationality").show(n=10, truncate=False)

If we want to select (multiple) **distinct** columns, we will use the <code>distinct</code> method.

In [0]:
fifa_df.select(["nationality"]).distinct().show(n=10, truncate=False)

In [0]:
fifa_df.select(["club", "nationality"]).distinct().show(n=10, truncate=False)

### **Find Duplicates (if any)**

In [0]:
print("The total number of duplicated ages are {:d} out of {:d}".
      format(fifa_df.count() - fifa_df.dropDuplicates(["age"]).count(), fifa_df.count()))

### **Filtering Data**
In order to filter data, according to the condition specified, we use the <code>filter</code> command. Here we are filtering our DataFrame based on the condition that <code>team_jersey_number</code> must be equal to <code>10</code> and then we are calculating how many records/rows are there in the filtered output.

In [0]:
fifa_df.filter(fifa_df.team_jersey_number==10).show(truncate=True)

In [0]:
print("The total number of players having jersey #10 are: {:d}".
      format(fifa_df.filter(fifa_df.team_jersey_number==10).count()))

### **Filtering Data (using multiple parameters)**
We can filter our data based on multiple logical conditions connected together by <code>AND</code> or <code>OR</code> operators.

In [0]:
# Filter Italian goalkeepers only
fifa_df.filter((fifa_df.team_position=="GK") & (fifa_df.nationality=="Italy")).show()

### **Sorting Data**

We can sort data using the <code>sort</code> method _or_ the <code>orderBy</code> method (which is just an alias of the former).

In [0]:
fifa_df.sort(["height_cm", "weight_kg"], ascending=[False, False]).show()
# Alternatively:
# fifa_df.orderBy(["height_cm", "weight_kg"], ascending=[False, False]).show()

### **Filtering + Sorting**

In [0]:
# Filter Italian goalkeepers only, and sort them by height in ascending order
fifa_df.filter((fifa_df.team_position=="GK") & (fifa_df.nationality=="Italy")).sort("height_cm").show()

### **Grouping Data**

In order to group data in a <code>DataFrame</code> on the basis of the values of one or more column, the <code>groupBy</code> operator is used.

In [0]:
# Let's group data by `age`
fifa_df.groupby(["age"]).count().show(10)

In [0]:
# Let's group data by `age` and sort it according to the resulting count in not-ascending order
fifa_df.groupby(["age"]).count().sort("count", ascending=False).show(10)

In [0]:
# Let's group data by `nationality` and `team_position`
fifa_df.groupby(["nationality", "team_position"]).count().sort("count", ascending=False).show(10)

In [0]:
# Let's group data by `nationality` and `team_position` (except "SUB" and "RES")
fifa_df.filter((fifa_df.team_position != "SUB") & (fifa_df.team_position != "RES")).groupby(["nationality", "team_position"]).count().sort("count", ascending=False).show(10)

### **Performing SQL Queries**

We can also pass SQL queries directly to any DataFrame, for that we need to create a table from the DataFrame using the <code>registerTempTable</code> method and then use  <code>sqlContext.sql()</code> to pass the SQL queries.

In [0]:
fifa_df.registerTempTable('fifa_table')
spark.sql('select * from fifa_table').show(10)

In [0]:
spark.sql('select distinct(club) from fifa_table').show(10)

## **4. Data Visualization**

A crucial step of the typical _exploratory analysis_ phase is to visually inspect data in order to get a sense of possible insights (e.g., _outliers_, _relationships_, _patterns_).

PySpark **does not** have any plotting functionality (yet). If we want to plot something, we **must** bring the data out of the Spark (_distributed_) Context and into our _local_ Python session, where we can deal with it using any of Python's many plotting libraries, such as <code>matplotlib</code> and <code>seaborn</code>. Please, refer to _Lecture 10_ available at this [link](https://github.com/gtolomei/python-for-datascience#Class-Schedule) for a deeper understanding of these libraries.

In [0]:
# The following import allows to use Python's plotting APIs: matplotlib and seaborn
import matplotlib.pyplot as plt
import seaborn as sns
# The following directive is to allow the inline visualization of generated plots
%matplotlib inline 

### **Move Data out from <code>SparkContext</code> into <code>Pandas</code>**

This can be achieved by calling the method <code>toPandas</code> on our original Spark DataFrame.

In [0]:
# Convert our `fifa_df` Spark DataFrame into the corresponding Pandas DataFrame
p_fifa_df = fifa_df.toPandas()

### **Plot <code>age</code> Distribution using <code>Pandas</code>' built-in plotting functions**


In [0]:
# Create a 1x1 figure
fig, ax = plt.subplots(1,1, figsize=(8,6))

_ = p_fifa_df.age.plot.hist(ax=ax, bins=10, color="lightblue", edgecolor='black', linewidth=1.2)
_ = p_fifa_df.age.plot.density(ax=ax, secondary_y=True, color="red")
_ = ax.set_xlabel("Age")
_ = ax.set_ylabel("Frequency")

### **Plot <code>age</code> Distribution using <code>matplotlib</code>'s plotting functions**

In [0]:
# Create a 1x1 figure
fig, ax = plt.subplots(1,1, figsize=(8,6))

_ = ax.hist(p_fifa_df.age, bins=10, color="lightblue", edgecolor='black', linewidth=1.2)
_ = ax
_ = ax.set_xlabel("Age")
_ = ax.set_ylabel("Frequency")

### **Plot <code>age</code> Distribution using <code>seaborn</code>'s plotting functions**

In [0]:
# Create a 1x1 figure
fig, ax = plt.subplots(1,1, figsize=(8,6))

_ = sns.distplot(p_fifa_df.age, 
                 bins=10, 
                 kde=True,
                 ax=ax, 
                 color="lightblue", 
                 hist_kws={"edgecolor":"k", "linewidth":1.2},
                 kde_kws={"color": "r", "lw": 1.5}
                 )
_ = ax.set_xlabel("Age")
_ = ax.set_ylabel("Density(%)")

### **Boxplot of a single variable**

In [0]:
# Create a 1x1 figure
fig, ax = plt.subplots(1,1, figsize=(6,4))

_ = sns.boxplot(x=p_fifa_df.height_cm, color="g")
# Alternatively:
#_ = sns.boxplot(x="height_cm", data=p_fifa_df, color="g")
_ = ax.set_xlabel("Height (cm)")

### **Boxplot combining 2 variables**

In [0]:
# Create a 1x1 figure
fig, ax = plt.subplots(1,1, figsize=(8,6))

_ = sns.boxplot(x="team_position", y="height_cm", data=p_fifa_df)
_ = ax.set_xlabel("Team Position")
_ = ax.set_xlabel("Height (cm)")

In [0]:
# Let's restrict the same plot above to the top-k position, escept for "SUB" and "RES"
k = 10
# Filter out `SUB` and `RES` players
filtered_df = p_fifa_df[(p_fifa_df.team_position != "SUB") & (p_fifa_df.team_position != "RES")]
# Find out what are the top-k roles
top_k_roles = filtered_df["team_position"].value_counts()[:k].index.values
print("The top-{:d} roles are: {:s}".format(k, ", ".join(top_k_roles)))
# Select only those top-k players
filtered_df = filtered_df[(filtered_df.team_position.isin(top_k_roles))]

In [0]:
# Create a 1x1 figure
fig, ax = plt.subplots(1,1, figsize=(8,6))

_ = sns.boxplot(x="team_position", y="height_cm", data=filtered_df)
_ = ax.set_xlabel("Team Position")
_ = ax.set_xlabel("Height (cm)")

### **Pairplot: Showing relationship between 2 or more variables**

In [0]:
_ = sns.pairplot(p_fifa_df[["height_cm", "weight_kg"]],
                 kind="reg",
                 diag_kind="hist", # hist or kde
                 diag_kws={"color":"r", "alpha":0.7, "edgecolor":"k"},     
                 height=3
                 )